In [3]:
import pandas as pd
import json
import os
import glob
import nltk
import string as st
import ast

In [9]:
df = pd.read_csv('./temp_data/enron_profanity_scores.csv')
sent_df = pd.read_csv('./temp_data/post_sentiment.csv')

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (30,36,37,40,53,56,57,67) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [23]:
merged_df = df.merge(sent_df, how="left", left_on='Email ID',right_on='filename')
merged_df.head()

,Email ID,Sender,Recipient,Date,Profanity Score,Unnamed: 0,index,text,subject,messageId,...,headers.eb32c2,headers.?,headers.your office (jessica ramiez),headers.(lavorato's office),headers.re,headers.(maureen x31808),headers.mark's office - eb3325,"headers.interconnect""",sentiment_text,sia_polarity_score
0,a6b28e2dede5b2b4fd0892d9b465679c,toddbart@aol.com,['kenneth.lay@enron.com'],2001-12-12T02:29:12.000Z,1.000000,142209,0,FUCK YOU>,(no subject),17661227.1075860840766.JavaMail.evans@thyme,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FUCK YOU,"{'neg': 0.778, 'neu': 0.222, 'pos': 0.0, 'comp..."
1,38fc72f9e701deba646512881b3497bd,brettlawler@hotmail.com,"['bubbad@wt.net', 'david.dickey@us.cgeyc.com',...",2000-12-13T01:57:00.000Z,1.000000,142706,0,"Hey Y'all,\n\nLess then 2 weekes before I'm ba...",Fwd: FW: Johnny Christmas Letter,8114562.1075854660857.JavaMail.evans@thyme,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hey Yall Less 2 weekes Im back Paradise Texas ...,"{'neg': 0.278, 'neu': 0.567, 'pos': 0.155, 'co..."
2,6b3a82ed0e2083c97d73556bbbfb0f70,stichavsky@pct-inc.com,"['baughman.travis@enron.com', 'beck.shantte@en...",2001-09-17T14:25:11.000Z,0.999982,128293,0,\n> I can't take credit for this but I though...,New Speech,23568905.1075840345672.JavaMail.evans@thyme,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I cant take credit I thought might find intere...,"{'neg': 0.228, 'neu': 0.627, 'pos': 0.145, 'co..."
3,5777f4f3ff0f07b5e803a96507fd7a3e,don.baughman@enron.com,"['richard.hrabal@enron.com', 'kayne.coulter@en...",2000-11-20T08:27:00.000Z,0.999974,160265,0,TOP 10 WAYS TO BE THE FUNNIEST GUY IN YOUR OFF...,Top 10,16449307.1075857881721.JavaMail.evans@thyme,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TOP 10 WAYS TO BE THE FUNNIEST GUY IN YOUR OFF...,"{'neg': 0.246, 'neu': 0.565, 'pos': 0.189, 'co..."
4,5fb15d479fd74b52cd25e606c115e085,chris.dorland@enron.com,['michael.cowan@enron.com'],2000-12-04T11:15:00.000Z,0.999970,240402,0,Did you where you clown suit to work?\n\nWhat ...,NaN,33313127.1075857312730.JavaMail.evans@thyme,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Did clown suit work What fucking shit show cd,"{'neg': 0.327, 'neu': 0.673, 'pos': 0.0, 'comp..."


In [24]:
merged_df['sia_pos'] = merged_df['sia_polarity_score'].apply(lambda row: ast.literal_eval(row)['pos'])
merged_df['sia_neg'] = merged_df['sia_polarity_score'].apply(lambda row: ast.literal_eval(row)['neg'])
merged_df['sia_neu'] = merged_df['sia_polarity_score'].apply(lambda row: ast.literal_eval(row)['neu'])
merged_df['sia_compound'] = merged_df['sia_polarity_score'].apply(lambda row: ast.literal_eval(row)['compound'])

In [25]:
col_slice = ['Sender', 'sia_pos', 'sia_neg', 'sia_neu', 'sia_compound']
slice_df = merged_df[col_slice]

In [26]:
slice_df.groupby(['Sender']).mean()
slice_df.groupby(['Sender']).max()
slice_df.groupby(['Sender']).min()

,sia_pos,sia_neg,sia_neu,sia_compound
Sender,,,,
amanda.rybarski@enron.com,0.000,0.558,0.442,-0.7430
benjamin.rogers@enron.com,0.000,0.419,0.581,-0.5574
beth.cherry@enform.com,0.179,0.054,0.407,-0.7783
brettlawler@hotmail.com,0.155,0.278,0.567,-0.9958
cara.semperger@enron.com,0.201,0.000,0.149,-0.1779
...,...,...,...,...
thinkguzman@yahoo.com,0.171,0.139,0.691,0.8076
todd.perry@enron.com,0.150,0.122,0.728,0.5606
toddbart@aol.com,0.000,0.325,0.222,-0.8176


In [35]:
top_five = slice_df.groupby(['Sender']).mean().sort_values(by='sia_neg', ascending=False)[0:5]
final_list = merged_df[merged_df['Sender'].isin(top_five.index)]
final_list.head()

,Email ID,Sender,Recipient,Date,Profanity Score,Unnamed: 0,index,text,subject,messageId,...,headers.re,headers.(maureen x31808),headers.mark's office - eb3325,"headers.interconnect""",sentiment_text,sia_polarity_score,sia_pos,sia_neg,sia_neu,sia_compound
11,12f4978096c3304604eb44754a8f59ed,kay.mann@enron.com,"['mreuter@kslaw.com', 'jkeffer@kslaw.com']",2001-03-25T15:27:00.000Z,0.999466,146278,0,"So how do you say ""bitch"" in Spanish?",NaN,29952283.1075845997380.JavaMail.evans@thyme,...,NaN,NaN,NaN,NaN,So say bitch Spanish,"{'neg': 0.576, 'neu': 0.424, 'pos': 0.0, 'comp...",0.000,0.576,0.424,-0.6222
17,63987840ea583a2452a30d0c3e2ca01f,daniel.diamond@enron.com,['martin.cuilla@enron.com'],2001-10-24T16:28:16.000Z,0.997282,218829,0,How bad does your ass hurt?,NaN,3486243.1075853088898.JavaMail.evans@thyme,...,NaN,NaN,NaN,NaN,How bad ass hurt,"{'neg': 0.667, 'neu': 0.095, 'pos': 0.238, 'co...",0.238,0.667,0.095,-0.6705
27,87eb112ec420c01bbb16ed62dd888da0,larry.campbell@enron.com,"['john.kinser@enron.com', 'robertmun@hotmail.c...",2001-04-18T15:26:00.000Z,0.987541,45009,0,---------------------- Forwarded by Larry F Ca...,Fwd: Re: given a dog a bath,21884837.1075857932439.JavaMail.evans@thyme,...,NaN,NaN,NaN,NaN,Forwarded Larry F CampbellNAEnron 04182001 032...,"{'neg': 0.124, 'neu': 0.716, 'pos': 0.159, 'co...",0.159,0.124,0.716,0.9338
41,575570e167f31e7b212d19d67be28e0d,mike.maggi@enron.com,['michelle.nelson@enron.com'],2001-11-26T17:58:51.000Z,0.970070,54255,0,where did you go you big jerk,NaN,15674097.1075862039862.JavaMail.evans@thyme,...,NaN,NaN,NaN,NaN,go big jerk,"{'neg': 0.545, 'neu': 0.455, 'pos': 0.0, 'comp...",0.000,0.545,0.455,-0.3400
50,056ad882ee6d71bb83e20748f2c814f5,michelle.nelson@enron.com,['mike.maggi@enron.com'],2001-11-26T19:09:15.000Z,0.950894,206154,0,you can't ignore me. i said to call me big jerk,NaN,22354421.1075862039236.JavaMail.evans@thyme,...,NaN,NaN,NaN,NaN,cant ignore said call big jerk,"{'neg': 0.282, 'neu': 0.47, 'pos': 0.248, 'com...",0.248,0.282,0.470,-0.0747


In [28]:
# for row in final_list.iterrows():
#     print('\n\n***************************\n\n\n')
#     print(row[1]['text'])

In [29]:
# raw search emails

query_word = 'fuck'
query_df = merged_df[merged_df.apply(lambda row: row.astype(str).str.contains(query_word, case=False).any(), axis=1)]
query_df.head()

,Email ID,Sender,Recipient,Date,Profanity Score,Unnamed: 0,index,text,subject,messageId,...,headers.re,headers.(maureen x31808),headers.mark's office - eb3325,"headers.interconnect""",sentiment_text,sia_polarity_score,sia_pos,sia_neg,sia_neu,sia_compound
0,a6b28e2dede5b2b4fd0892d9b465679c,toddbart@aol.com,['kenneth.lay@enron.com'],2001-12-12T02:29:12.000Z,1.000000,142209,0,FUCK YOU>,(no subject),17661227.1075860840766.JavaMail.evans@thyme,...,NaN,NaN,NaN,NaN,FUCK YOU,"{'neg': 0.778, 'neu': 0.222, 'pos': 0.0, 'comp...",0.000,0.778,0.222,-0.5423
1,38fc72f9e701deba646512881b3497bd,brettlawler@hotmail.com,"['bubbad@wt.net', 'david.dickey@us.cgeyc.com',...",2000-12-13T01:57:00.000Z,1.000000,142706,0,"Hey Y'all,\n\nLess then 2 weekes before I'm ba...",Fwd: FW: Johnny Christmas Letter,8114562.1075854660857.JavaMail.evans@thyme,...,NaN,NaN,NaN,NaN,Hey Yall Less 2 weekes Im back Paradise Texas ...,"{'neg': 0.278, 'neu': 0.567, 'pos': 0.155, 'co...",0.155,0.278,0.567,-0.9958
2,6b3a82ed0e2083c97d73556bbbfb0f70,stichavsky@pct-inc.com,"['baughman.travis@enron.com', 'beck.shantte@en...",2001-09-17T14:25:11.000Z,0.999982,128293,0,\n> I can't take credit for this but I though...,New Speech,23568905.1075840345672.JavaMail.evans@thyme,...,NaN,NaN,NaN,NaN,I cant take credit I thought might find intere...,"{'neg': 0.228, 'neu': 0.627, 'pos': 0.145, 'co...",0.145,0.228,0.627,-0.9801
3,5777f4f3ff0f07b5e803a96507fd7a3e,don.baughman@enron.com,"['richard.hrabal@enron.com', 'kayne.coulter@en...",2000-11-20T08:27:00.000Z,0.999974,160265,0,TOP 10 WAYS TO BE THE FUNNIEST GUY IN YOUR OFF...,Top 10,16449307.1075857881721.JavaMail.evans@thyme,...,NaN,NaN,NaN,NaN,TOP 10 WAYS TO BE THE FUNNIEST GUY IN YOUR OFF...,"{'neg': 0.246, 'neu': 0.565, 'pos': 0.189, 'co...",0.189,0.246,0.565,-0.9236
4,5fb15d479fd74b52cd25e606c115e085,chris.dorland@enron.com,['michael.cowan@enron.com'],2000-12-04T11:15:00.000Z,0.999970,240402,0,Did you where you clown suit to work?\n\nWhat ...,NaN,33313127.1075857312730.JavaMail.evans@thyme,...,NaN,NaN,NaN,NaN,Did clown suit work What fucking shit show cd,"{'neg': 0.327, 'neu': 0.673, 'pos': 0.0, 'comp...",0.000,0.327,0.673,-0.5984


In [37]:
full_racist_email_df = pd.read_csv('./temp_data/racist_scan.csv')
full_racist_email_df.shape
# full_racist_email_df.to_csv('./temp_data/racist_scan_output.csv')

(417, 74)

In [22]:
# for email in full_racist_email_df[0:10].iterrows():
#     print(email[1]['text'])